이 문서는 Google Colaboratory에 카타고를 세팅하고 리지나 사바키(또는 다른 GTP 엔진을 지원하는 도구)와 연결하는 방법을 설명합니다.

# 1단계 : Colab 서버에서 KataGo와 SSH 실행하기

처음 실행하기 전에 NGROK_TOKEN 값을 입력해 주어야 합니다.

구글계정마다 값이 다르기 때문에 콜랩을 실행하시는 컴퓨터의 구글계정이 바뀐 경우는 재입력 하셔야 합니다.

아래의 링크를 눌러서 이동하여 맨 위에 Copy 버튼을 누르시면 토큰값이 복사됩니다.

[https://dashboard.ngrok.com/auth/your-authtoken](https://dashboard.ngrok.com/auth/your-authtoken).

그 다음 아래 스크립트 영역 맨 윗줄의 입력하는 곳에 붙여넣기 하시면 됩니다.

USER_PASSWORD 값은 원하시는 값으로 바꿔주세요.

(이 경우 나중에 PC의 리지 설정에서도 똑같이 바꿔주어야 합니다 ⇒ 마지막에 나오는 명령어 안내 참조)

TeslaT4 GPU가 연결된 경우에 60블럭 가중치를 사용하시려면 T4_WEIGHT 값을 60b로 바꾸시면 됩니다.(권장하지는 않습니다.)

그 다음에는 아래에서 실행버튼(동그란 아이콘)을 눌러서 실행하시면 됩니다.

그리고, 구글드라이브 연결을 위한 권한획득 절차가 있습니다.
중간에 나오는 안내에 따라 실행하시면 됩니다.

전체 스크립트가 실행되는 데 1~2분 정도 걸립니다.
참고로, 자동으로 서버에 할당된 GPU가 CUDA를 지원하지 않는 경우 처음 리지를 실행했을 때 OpenCL 튜닝 절차가 진행될 수도 있습니다.

CUDA 버전이 선택된 경우 대략 NVIDIA RTX 2060 정도의 성능이 나오는 듯 합니다.

In [ ]:
# SSH configuration
NGROK_TOKEN = 'mytoken'
USER_PASSWORD = 'mypassword'

# KataGo configuration
KATAGO_BACKEND = 'AUTO'
T4_WEIGHT = '40b'
ELSE_WEIGHT = '20b'

# Constant
KATAGO_COLAB_REPOSITORY_URL = 'https://github.com/matobataketoshi/katago-colab.git'
KATAGO_CONFIG_FILE = '/content/katago-colab/config/gtp_colab.cfg'
KATAGO_TUNING_DIR = '/content/katago-colab/opencltuning'
KATAGO_RESOURCE_FILE = '/content/katago-colab/colab-resource/external-resource.json'
SSH_INFO_FILE_NAME = 'colab-katago-ssh.json'
SSH_INFO_DIR = '/content/drive/MyDrive'

# Install useful stuff
!echo "필요한 라이브러리를 설치합니다."
!apt-get update 1>/dev/null
!apt-get install --yes ssh screen nano htop ranger git libzip4 1>/dev/null
!wget -q https://github.com/wonsiks/katago-colab/releases/download/v1.9.1/libzip.so.5.0 -O /usr/lib/x86_64-linux-gnu/libzip.so.5
!pip install -U -q PyDrive 1>/dev/null

import subprocess
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import requests
import json
import os
from re import sub

def get_katago_resource(katago_resource_file):
    with open(katago_resource_file, 'r') as f:
        resource = json.load(f)
    return resource

def save_ssh_info(dir, file_name):
    file_path = dir + '/' + file_name
    ssh_option = create_ssh_option()
    file_id = get_file_id_in_drive(file_path)
    if file_id is None:
        file_id = upload_ssh_option_with_pydrive(file_name, ssh_option)
    else:
        ssh_option['fileId'] = file_id
        write_ssh_option(file_path, ssh_option)
    print('================================================================')
    print('SSH_INFO_GOOGLE_DRIVE_FILE_ID: {}'.format(file_id))
    print('================================================================')
    return file_id

def create_ssh_option():
    ssh_option = None
    try:
        r = requests.get('http://localhost:4040/api/tunnels')
        raw_ssh = r.json()['tunnels'][0]['public_url']
        ssh_args = (sub("tcp://", "", raw_ssh)).split(':')
        ssh_option = {
            'host': ssh_args[0],
            'port': int(ssh_args[1]),
            'user': 'root'
        }
    except Exception as e:
        print('Failed to create SSH options. Make sure that the ngrok token is set correctly.')
        raise e
    return ssh_option

def get_file_id_in_drive(file_path):
    file_id = None
    if (os.path.exists(file_path)):
        try:
            with open(file_path, 'r') as f:
                ssh_option = json.load(f)
                file_id = ssh_option.get('fileId')
        except Exception as e:
            file_id = None
    return file_id

def write_ssh_option(file_path, ssh_option):
    print('Write SSH option: {}'.format(file_path))
    with open(file_path, 'w') as f:
        f.write(json.dumps(ssh_option))
 
def upload_ssh_option_with_pydrive(file_name, ssh_option):
    print('Authenticate to Google Drive')
    drive = auth_google_drive()
    file_metadata = {
        'title': file_name,
        'mimeType': 'application/json'
    }
    file_list = drive.ListFile({'q': 'title="' + file_name + '" and trashed=False'}).GetList()
    if (len(file_list) > 0):
        file_metadata['id'] = file_list[0]['id']
    ssh_option_file = drive.CreateFile(file_metadata)
    ssh_option_file.Upload()
    file_id = ssh_option_file.get('id')
    ssh_option['fileId'] = file_id
    ssh_option_file.SetContentString(json.dumps(ssh_option))
    ssh_option_file.Upload()
    ssh_option_file.InsertPermission({'type': 'anyone', 'value': 'anyone', 'role': 'reader'})
    return file_id

def auth_google_drive():
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    return drive

# Get GPU name
gpu_name = str(subprocess.check_output("nvidia-smi -q | grep \"Product Name\" | cut -d\":\" -f2 | tr -cd '[:alnum:]._-'", shell=True), encoding='utf-8')
print('GPU: {}'.format(gpu_name))

# Select backend type
if KATAGO_BACKEND == "AUTO":
  if gpu_name == "TeslaT4":
    KATAGO_BACKEND = "TensorRT"
  else:
    KATAGO_BACKEND = "OPENCL"    
print('Using KataGo Binery: {}'.format(KATAGO_BACKEND))

# Select weight file
if gpu_name == "TeslaT4":
  WEIGHT_FILE = T4_WEIGHT
else:
  WEIGHT_FILE = ELSE_WEIGHT
print('Using KataGo Weight: {}'.format(WEIGHT_FILE))

%cd /content
# Clone katago-colab
!echo "Git로부터 필요한 파일들을 받아옵니다."
!rm -rf katago-colab
!git clone $KATAGO_COLAB_REPOSITORY_URL 1>/dev/null

# Get URLs of external resources
katago_resource = get_katago_resource(KATAGO_RESOURCE_FILE)
ngrok_url = katago_resource['ngrok']
katago_url = katago_resource[KATAGO_BACKEND]
weight_url = katago_resource[WEIGHT_FILE]

if KATAGO_BACKEND == "TensorRT":
  !echo "TensorRT 구동에 필요한 파일들을 추가로 설치합니다."

  %cd /content
  !wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/libcudnn8_8.2.1.32-1+cuda11.3_amd64.deb
  !dpkg -i libcudnn8_8.2.1.32-1+cuda11.3_amd64.deb

  !wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/libnvinfer8_8.2.2-1+cuda11.4_amd64.deb
  !dpkg -i libnvinfer8_8.2.2-1+cuda11.4_amd64.deb
  !wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/libnvonnxparsers8_8.2.2-1+cuda11.4_amd64.deb
  !dpkg -i libnvonnxparsers8_8.2.2-1+cuda11.4_amd64.deb
  !wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/libnvparsers8_8.2.2-1+cuda11.4_amd64.deb
  !dpkg -i libnvparsers8_8.2.2-1+cuda11.4_amd64.deb

# Download ngrok
!wget -q $ngrok_url -O ngrok
!chmod +x /content/ngrok
 
# SSH setting
!echo "root:$USER_PASSWORD" | chpasswd
!echo "PasswordAuthentication yes" > /etc/ssh/sshd_config
!echo "PermitUserEnvironment yes" >> /etc/ssh/sshd_config
!echo "PermitRootLogin yes" >> /etc/ssh/sshd_config

!mkdir -p /root/.ssh
!service ssh restart > /dev/null
 
# Run ngrok
!echo "ngrok를 실행합니다."
get_ipython().system_raw('./ngrok authtoken $NGROK_TOKEN && ./ngrok tcp 22 &')
!sleep 5

# Save SSH info to Google Drive
!echo "구글드라이브에 SSH 접속정보를 저장합니다."
SSH_INFO_GOOGLE_DRIVE_FILE_ID = save_ssh_info(SSH_INFO_DIR, SSH_INFO_FILE_NAME)

# Download KataGo binary
!echo "카타고 실행파일을 받아옵니다."
!wget -q "$katago_url" -O katago
!chmod +x /content/katago
!/content/katago version

# Put KataGo tuning files
!mkdir -p /root/.katago/
if KATAGO_BACKEND == "OPENCL":
  !cp -r $KATAGO_TUNING_DIR /root/.katago/

# Download a network file of KataGo
!echo "카타고 가중치를 받아옵니다."
!rm -rf weight.bin.gz
!wget -q "$weight_url" -O "weight.bin.gz"

# Copy enviroment variable for SSH
with open('/root/.ssh/environment','w') as f:
    for key, value in os.environ.items():
        print('{}={}'.format(key, value), file=f)

!echo -e "\n[KataGo Config]" 
!cat $KATAGO_CONFIG_FILE
!echo -e "\n"

!echo "연결 준비가 완료되었습니다! 이제 리지를 연결하실 수 있습니다..."
!echo "리지 엔진구동 명령어 : ./colab-katago-gd.exe $SSH_INFO_GOOGLE_DRIVE_FILE_ID $USER_PASSWORD"

# 2단계 : Colab KataGo를 사바키 또는 리지에 연결하기<br>(GOOGLE_DRIVE_ID와 사용자암호는 직접 수정하셔야 합니다.)

아래 링크에서 배포 패키지를 받아서 설치하시고, 리지 엔진구동 명령어를 수정하여 사용하시기 바랍니다.

http://naver.me/IDBjK9BH